In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from HelperFunctions import *
from PtOccupancyFunctions import *
import scipy.integrate
import pprint
import re

In [ ]:
path = "/home/helge/Applications/matsim/matsim_results_ssd/PeriodicBC/8DrtCap/OwnIntermodalAccEgr/OptDrtCount/new/Uniform5e4Reqs1800EllFine/"

In [ ]:
def getBimDirsFleetSizes(directory):
    result = []
    sub_dirs = []
    sdirs = [
        sdir.path
        for sdir in
            os.scandir(directory)
        if sdir.is_dir() and "drt" in sdir.name
    ]

    for sdir in sorted(sdirs, key=lambda x: int(re.search("\d+",x.split("/")[-1])[0])):
        subresult = {}
        subresult["root"] = sdir
        for root, subdirs, files in os.walk(sdir):
            for file in files:
                if file=="0.trips.csv.gz" and "unimodal" not in root:
                    subresult["trips"] = os.path.join(root, file)
                if file=="0.vehicleDistanceStats_drt.csv" and "unimodal" not in root:
                    subresult["drt_dists"] = os.path.join(root, file)
                if file=="trip_success.csv.gz" and "unimodal" not in root:
                    subresult["trip_success"] = os.path.join(root, file)
                if file=="0.CummulativePtDistance.txt" and "unimodal" not in root:
                    subresult["pt_dist"] = os.path.join(root, file)
                if file=="0.drt_occupancy_time_profiles_drt.txt" and "unimodal" not in root:
                    subresult["drt_occupancy"] = os.path.join(root, file)
                if file=="0.occupancyAnalysis.txt" and "unimodal" not in root:
                    subresult["pt_occupancy"] = os.path.join(root, file)
                if file=="0.drt_trips_drt.csv" and "unimodal" not in root:
                    subresult["drt_trips"] = os.path.join(root, file)
                if file=="0.drt_detours_drt.csv" and "unimodal" not in root:
                    subresult["drt_detours"] = os.path.join(root, file)
                if file=="ph_modestats.txt" and "unimodal" not in root:
                    subresult["ph_modestats"] = os.path.join(root, file)
                if file=="pkm_modestats.txt" and "unimodal" not in root:
                    subresult["pkm_modestats"] = os.path.join(root, file)
                    
        result.append(subresult)
    
    return result


def getDir(path, directory):
    for root, subdirs, files in os.walk(path):
        for subdir in subdirs:
            if subdir == directory:
                return os.path.join(root, subdir)

bim_dirs = getBimDirsFleetSizes(path)

In [ ]:
pprint.pprint(bim_dirs)
# print('\n---\n')
# pprint.pprint(uni_dirs)
# print('\n---\n')
# pprint.pprint(car_dir)

In [ ]:
def getTrips(paths, mode):
    if mode == "bimodal":
        columns=["person","trav_time","wait_time","traveled_distance","modes"]
    elif mode == "unimodal":
        columns=["person","trav_time","wait_time"]
    elif mode == "car":
        columns=["person", "traveled_distance", "trav_time"]
    path = paths["trips"]
    df = pd.read_csv(path, sep=";").loc[:,columns]
    df["trav_time"] = df["trav_time"].apply(timestmp2sec)
    df.set_index("person", inplace=True)
    if mode=="car":
        df = df.groupby("person").agg(
            {
                "trav_time": np.sum,
                "traveled_distance": np.sum
            }
        )
        return df
    df["wait_time"] = df["wait_time"].apply(timestmp2sec)
    if mode=="unimodal":
        df = df.groupby("person").agg(
            {
                "trav_time": np.sum,
                "wait_time": np.sum,
            }
        )
        return df
    if mode == "bimodal":
        df = df.groupby("person").agg(
            {
                "trav_time": np.sum,
                "wait_time": np.sum,
                "traveled_distance": np.sum,
                "modes": combineModesSeriesStr,
            }
        )
        return df
    
def getFleetSize(path):
    return re.search("\d+", path.split("/")[-1])[0]

def getEll(path):
    return path.split("/")[-1].split("_")[-1]

def getModeStats(paths, columns_ph, columns_pkm):
    path_ph = paths["ph_modestats"]
    path_pkm = paths["pkm_modestats"]
    df_ph = pd.read_csv(path_ph, sep='\t').loc[:,columns_ph]
    df_pkm = pd.read_csv(path_pkm, sep='\t').loc[:,columns_pkm]
    return df_ph.to_numpy()[0], df_pkm.to_numpy()[0]

def getDrtVehicleDistances(paths):
    path = paths["drt_dists"]
    df = pd.read_csv(path, sep=";")["drivenDistance_m"]
    return df

def getTripSuccess(paths):
    path = paths["trip_success"]
    df = pd.read_csv(path, sep=";", index_col="personId")
    return df

def getCummulativePtDistance(paths):
    path = paths["pt_dist"]
    df = pd.read_csv(path).values[0, 0]
    return df

def getDrtOccupandyAndStandingFrac(paths, exclude_empty_vehicles, count_idle_vehicles=False):
    path = paths["drt_occupancy"]
    df = pd.read_csv(path, sep="\t")
    df["time"] = df["time"].apply(timestmphm2sec)
    drt_occ, drt_deviation = getAverageOcc(
        df.drop(columns="time"), exclude_empty_vehicles=exclude_empty_vehicles, count_idle_vehicles=count_idle_vehicles
    )
    drt_standing_frac = getStandingFraction(df.drop(columns="time"))
    return drt_occ, drt_standing_frac

def getPtOccupancy(paths):
    path = paths["pt_occupancy"]
    av_pt_occ, av_pt_occ_sq, n_pt = getPtOccupancies(
        path, 600
    )
    t_av_pt_occ_av = getAverageTimeSeries(av_pt_occ)
    sigma = np.sqrt(n_pt / (n_pt - 1)) * np.sqrt(
        av_pt_occ_sq - av_pt_occ ** 2
    )
    t_av_pt_occ_sigma = getAverageTimeSeries(sigma)
    return t_av_pt_occ_av, t_av_pt_occ_sigma

def getDrtTrips(paths):
    path = paths["drt_trips"]
    df = pd.read_csv(path, sep=";").loc[:, ["personId", "travelDistance_m", "waitTime"]]
    df.set_index("personId", inplace=True)
    df = df.groupby("personId").agg(
        {
            "waitTime": np.sum,
            "travelDistance_m": np.sum,
        }
    )
    return df

def getDrtDetours(paths):
    path = paths["drt_detours"]
    df = pd.read_csv(path, sep=";").loc[
        :, "distanceDetour"
    ]
    df = df[df < 10]
    return df

# display(getTrips(bim_dirs[0], "bimodal"))

In [ ]:
fleet_list = np.empty(21)
cummulative_drt_bimodal = np.empty(21)
cummulative_train_bimodal = np.empty(21)
cummulative_car_bimodal = np.empty(21)
av_drt_occs = np.empty(21)
av_drt_occs_non_empty = np.empty(21)
av_drt_occs_non_standing = np.empty(21)
wait_times = np.empty(21)
wait_times_drt = np.empty(21)
wait_times_served_only = np.empty(21)
wait_times_drt_unimodal_legs = np.empty(21)
wait_times_drt_bimodal_legs = np.empty(21)
serviced_fracts = np.empty(21)
av_pt_occs = np.empty(21)
bi_or_unimodal = np.empty((21,2))
drt_pt_person_km = np.empty((21,2))
av_detours = np.empty(21)
drt_wait_times = np.empty(21)
pt_wait_times = np.empty(21)
tt_ratios_beyond_lcut = np.empty(21)
tt_bimodal = np.empty(21)
# rt_bimodal = np.empty(21)
standing_fractions = np.empty(21)
ph_modestats = np.empty((21,5))
pkm_modestats = np.empty((21,3))

ph_columns=["drt_travel", "drt_wait", "pt_travel", "pt_wait", "walk_travel"]
pkm_columns=["drt", "pt", "walk"]

In [ ]:
for i,dic in enumerate(bim_dirs):
    fleet_list[i] = getFleetSize(dic["root"])
    print("Fleet Size: ", fleet_list[i])
    trips = getTrips(dic, "bimodal")
    drt_trips = getDrtTrips(dic)
#     drt_veh_dists = getDrtVehicleDistances(dic)
    trip_succ = getTripSuccess(dic)
    av_drt_occs_non_standing[i], standing_fractions[i] = getDrtOccupandyAndStandingFrac(dic, False)
    av_drt_occs_non_empty[i], _ = getDrtOccupandyAndStandingFrac(dic, True)
    av_drt_occs[i], _ = getDrtOccupandyAndStandingFrac(dic, False, True)
#     av_drt_occs_non_empty[i] = drt_occupancy_non_empty
#     av_drt_occs_non_standing[i] = drt_occupancy_non_standing
    av_pt_occs[i], pt_occ_variance = getPtOccupancy(dic)
    drt_detours = getDrtDetours(dic)
#     pt_dist = getCummulativePtDistance(dic)
#     ph_modestats[i], pkm_modestats[i] = getModeStats(dic, ph_columns, pkm_columns)
    
#     drt_pt_person_km[i,0] = drt_trips["travelDistance_m"].sum()
#     drt_pt_person_km[i,1] = trips["traveled_distance"].sum()-drt_pt_person_km[i,0]
    serviced_fracts[i] = len(trip_succ[trip_succ["tripSuccess"]]) / len(trip_succ)
#     display(trip_succ[~trip_succ["tripSuccess"]])
    print("Servability: ", serviced_fracts[i])
# #     display(len(dists_car.sort_index()))
# #     display(trip_succ)
#     dists_car_for_rejected = dists_car.loc[
#         trip_succ[~trip_succ["tripSuccess"]].index.to_numpy()
#     ]
#     cummulative_drt_bimodal[i] = drt_veh_dists.sum()
#     cummulative_car_bimodal[i] = dists_car_for_rejected.sum()
#     cummulative_train_bimodal[i] = pt_dist
# #     av_trav_time_bimodal = trav_times[pt_in_trips]
#     pt_in_trips = trips["modes"].str.contains("pt")
#     pt_in_trips_idx = pt_in_trips[pt_in_trips]
#     pt_not_in_trips_idx = pt_in_trips[~pt_in_trips]
#     count_pt_in_trips = pt_in_trips.value_counts()
#     bi_or_unimodal[i,0] = count_pt_in_trips[False]
#     bi_or_unimodal[i,1] = count_pt_in_trips[True]
    av_detours[i] = drt_detours.mean()
    tt_bimodal[i] = trips["trav_time"].mean()
# #     rt_bimodal[i] = (trips["trav_time"] - trips["wait_time"]).mean()
    wait_times[i] = trips["wait_time"].mean()
    wait_times_drt[i] = drt_trips["waitTime"].mean()
    
#     idx_served = trip_succ[trip_succ["tripSuccess"]].index
#     wait_times_served_only[i] = trips.loc[idx_served, "wait_time"].mean()
    
#     drt_wait_times[i] = drt_trips["waitTime"].mean()
# #     wait_times_drt_unimodal_legs[i] = drt_trips.loc[~pt_in_trips,"waitTime"].mean()
# #     wait_times_drt_bimodal_legs[i] = drt_trips.loc[pt_in_trips,"waitTime"].mean()
#     wait_times_drt_unimodal_legs[i] = drt_trips.drop(pt_in_trips_idx, errors='ignore')["waitTime"].mean()
#     wait_times_drt_bimodal_legs[i] = drt_trips.drop(pt_not_in_trips_idx, errors='ignore')["waitTime"].mean()
#     pt_wait_series = (trips["wait_time"] - drt_trips["waitTime"]).fillna(0)
#     pt_wait_times[i] = pt_wait_series[pt_wait_series != 0].mean()
#     idx_pt_in_trips = pt_in_trips[pt_in_trips].index
#     tt_ratios_beyond_lcut[i] = trips["trav_time"][idx_pt_in_trips].sum()/trips_car["trav_time"][idx_pt_in_trips].sum()
    print("-----")

# bi_or_unimodal = pd.DataFrame(bi_or_unimodal, index=ell_list, columns=["Unimodal","Bimodal"])
# drt_pt_person_km = pd.DataFrame(drt_pt_person_km, index=ell_list, columns=["DRT","PT"])
# pkm_modestats = pd.DataFrame(pkm_modestats, index=ell_list, columns=pkm_columns)
# ph_modestats = pd.DataFrame(ph_modestats, index=ell_list, columns=ph_columns)

In [ ]:
fig, ax = plt.subplots(constrained_layout=True)

ax.plot(fleet_list, serviced_fracts, 'o-')
ax.set_ylim(0,1.1)

plt.show()

In [ ]:
fig, ax = plt.subplots(constrained_layout=True)

ax.plot(fleet_list, av_drt_occs, 'o-')

plt.show()

In [ ]:
fig, ax = plt.subplots(constrained_layout=True)

ax.plot(fleet_list, av_detours, 'o-')

plt.show()

In [ ]:
fig, ax = plt.subplots(constrained_layout=True)

#Normalize by TT for private car
ax.plot(fleet_list, tt_bimodal, 'o-')

plt.show()

In [ ]:
fig, ax = plt.subplots(constrained_layout=True)

#Normalize by TT for private car
ax.plot(fleet_list, wait_times, 'o-')

plt.show()

In [ ]:
fig, ax = plt.subplots(constrained_layout=True)

ax.plot(fleet_list, standing_fractions, 'o-')

plt.show()